In [219]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Token
import re
import numpy as np

def test_5w1h(text):
    nlp = spacy.load('ja_ginza')

    matcher = Matcher(nlp.vocab)

    if not Token.has_extension("_5w1h"):
        Token.set_extension("_5w1h", default='None')


    
    Where_pattern1 = [{"POS":{"REGEX":"NOUN|PROPN|PRON"},"ENT_TYPE":{"REGEX":"Country|Province"}}]
    Where_pattern2 = [{"TEXT":{"REGEX":"Amazon"}}]
    Where_pattern3 = [{"TEXT":"中"},{"TEXT":"に"}]
    
    When_pattern1 = [{"TEXT":{"REGEX":"今日|昨日|おととい|明日|先日|明後日|今回|前回|後ほど|その後"}}]
    When_pattern2 = [{"POS":"NUM"},{"TEXT":{"REGEX":"時|日|月|年"},"LENGTH":1}]
    When_pattern3 = [{"TEXT":{"REGEX":"今"},"LENGTH":1}]
    When_pattern4 = [{"TEXT":"この"},{"TEXT":{"REGEX":"後|前"}}] 
    
    Why_pattern1 = [{"POS":{"REGEX":"VERB|ADJ"},"OP":"+"},{"POS":{"REGEX":"AUX"},"OP":"?"},{"TEXT":"から"}]

    How_pattern1 = [{"POS":{"REGEX":"VERB|ADJ|NOUN"},"DEP":{"REGEX":"ROOT|punct"}}]
    How_pattern2 = [{"POS":{"REGEX":"VERB|ADJ"},"OP":"+"},{"DEP":{"REGEX":"ROOT|cc"}}]
    How_pattern3 = [{"POS":{"REGEX":"VERB|ADJ|NOUN"},"OP":"+"},{"POS":{"REGEX":"AUX|ADP"},"OP":"?"},{"DEP":{"REGEX":"ROOT"}}]
    How_pattern4 = [{"POS":{"REGEX":"VERB|ADJ"},"OP":"+"},{"OP":"?"},{"POS":"PUNCT"}]
    How_pattern5 = [{"POS":{"REGEX":"VERB|ADJ|AUX"},"OP":"*"},{"TEXT":"の"},{"TEXT":"に"}]
    #How_pattern6 = [{"DEP":"advcl"},{"DEP":"mark"}]
    How_pattern7 = [{"POS":"VERB","OP":"?"},{"POS":"AUX","OP":"+"},{"TEXT":"が","POS":"CCONJ"},{"POS":"PUNCT","OP":"?"}]
    How_pattern8 = [{"POS":"VERB"},{"POS":"AUX"},{"TEXT":"ところ","POS":"NOUN"}]
    How_pattern9 = [{"POS":"VERB","DEP":"advcl"},{"TEXT":"し","POS":"CCONJ"}]
    How_pattern10 = [{"POS":{"REGEX":"VERB|ADJ"}},{"TEXT":"ん","OP":"?"},{"TEXT":"です","OP":"?"},{"LEMMA":"けれど"},{"TEXT":"も","OP":"?"},{"TEXT":"、","OP":"?"}]
    How_pattern11 = [{"POS":"AUX"},{"POS":{"REGEX":"PUNCT"}}]
    How_pattern12 = [{"POS":{"REGEX":"VERB|ADJ|AUX"},"OP":"*"},{"TEXT":"けれど"},{"TEXT":"も"}]
    How_pattern13 = [{"POS":"VERB","OP":"?"},{"POS":"AUX","OP":"+"},{"TEXT":"と"},{"POS":"PUNCT","OP":"?"}]
    How_pattern14 = [{"POS":"AUX"},{"TEXT":"が","POS":{"REGEX":"CCONJ"}}]

    Who_pattern1 = [{"TEXT":"の","OP":"?"},{"POS":"NOUN","DEP":"compound","OP":"*"},{"POS":{"REGEX":"NOUN|PRON|PROPN"},"DEP":{"REGEX":"iobj|obl|nsubj"},"TAG":{"NOT_IN":["名詞-普通名詞-助数詞可能"]},"TEXT":{"NOT_IN":["幾つ"]}},{"TEXT":{"REGEX":"が|は|も"}},{"TEXT":{"REGEX":"です|ね|、"},"OP":"*"}]
    Who_pattern2 = [{"DEP":{"REGEX":"amod|advmod|acl"},"OP":"+"},{"TEXT":"ところ","DEP":{"REGEX":"compound"}}]
    Who_pattern3 = [{"POS":{"REGEX":"NOUN|PRON|PROPN"},"DEP":{"REGEX":"iobj|obl|nsubj"}},{"TEXT":{"REGEX":"に"}},{"TEXT":{"REGEX":"は"}}]
    Who_pattern4 = [{"POS":{"REGEX":"NOUN|PRON|PROPN"},"DEP":{"REGEX":"obl|nmod|dep"},"TEXT":{"NOT_IN":["幾つ"]},"TAG":{"NOT_IN":["名詞-普通名詞-助数詞可能"]}},{"TEXT":{"REGEX":"が|は|も|って"}}]
    Who_pattern5 = [{"TEXT":"こと","DEP":"compound"},{"TEXT":{"REGEX":"が|は|も"}}]
    Who_pattern6 = [{"POS":"NUM","OP":"!"},{"POS":{"REGEX":"NOUN|PRON|PROPN"},"DEP":{"REGEX":"iobj|obl|nsubj"},"TAG":"名詞-普通名詞-助数詞可能"},{"TEXT":{"REGEX":"が|は|も"}}]
    Who_pattern7 = [{"POS":{"REGEX":"VERB|ADJ"},"OP":"+"},{"POS":"NOUN","OP":"?"},{"TEXT":"の"},{"TEXT":"が"}]
    Who_pattern8 = [{"TEXT":"と"},{"TEXT":"いう"},{"TEXT":"の"},{"TEXT":"は"}]

    What_pattern1 = [{"POS":{"REGEX":"NOUN|PRON|PROPN"},"DEP":{"REGEX":"obl|obj|iobj"}},{"TEXT":{"REGEX":"を"}}]
    What_pattern2 = [{"POS":{"REGEX":"SYM"},"DEP":{"REGEX":"dep"}},{"TEXT":{"REGEX":"を"}}]


    Mod_pattern1 = [{"DEP":{"REGEX":"amod|advmod|nmod|case|obl|case|acl|aux|det|nsubj|dep|mark|compound|nummod|advcl|iobj|det|obj"}}]
    Mod_pattern2 = [{"POS":{"REGEX":"NOUN"},"DEP":{"REGEX":"nmod"}},{"POS":"ADP"}]
    Mod_pattern3 = [{"TEXT":"いつ"}]
    Mod_pattern4 = [{"POS":{"REGEX":"NOUN"},"DEP":"iobj"}]


    def add_label(matcher, doc, id, matches):
        l = list(matches[id])
        for t in doc[l[1]:l[2]]:
            if t._._5w1h == 'None' or t._._5w1h == "Mod":
                t._._5w1h = nlp.vocab.strings[l[0]]


    def add_right(matcher, doc, id, matches):
        l = list(matches[id])
        tag =  nlp.vocab.strings[l[0]]
        end = l[-1]
        
        if  end != len(doc):
            while doc[end].head.i == l[-1]-1:

                end = end + 1
                if end == len(doc):
                    break


        l[-1] = end
 
        if (re.search("ので|だから",doc[l[-1]-2:l[-1]].text) or re.search("ので、|だから、",doc[l[-1]-3:l[-1]].text) or re.search("ため",doc[l[-1]-1:l[-1]].text) or re.search("ため、",doc[l[-1]-2:l[-1]].text)) and doc[l[-1]].pos_ != "ADP":
            tag = "Why" 

        for t in doc[l[1]:l[2]]:
            if t._._5w1h == 'None' or t._._5w1h == "Mod" or tag == "Why":
                t._._5w1h = tag

        matches[id] = tuple(l)

    def add_right_left(matcher, doc, id, matches):
        l = list(matches[id])
        tag =  nlp.vocab.strings[l[0]]
        end = l[-1]
        start = l[1]
        
        if  end != len(doc):
            while doc[end].head.i == l[-1]-1:

                end = end + 1
                if end == len(doc):
                    break


        if start != 0:
            while doc[start].head.i == l[1]:
                start = start - 1
                if start == 0:
                    break

        l[1] = start
        l[-1] = end

        if (re.search("ので|だから",doc[l[-1]-2:l[-1]].text) or re.search("ので、|だから、",doc[l[-1]-3:l[-1]].text) 
            or re.search("ため",doc[l[-1]-1:l[-1]].text) or re.search("ため、",doc[l[-1]-2:l[-1]].text)) and doc[l[-1]].pos_ != "ADP":
            tag = "Why" 
          
        for t in doc[l[1]:l[2]]:
            if t._._5w1h == 'None' or t._._5w1h == "Mod" or tag == "Why" :
                t._._5w1h = tag

        matches[id] = tuple(l)


    matcher.add("When",add_right, When_pattern1,When_pattern2,When_pattern3,When_pattern4)
    matcher.add("Where", add_right, Where_pattern1,Where_pattern2,Where_pattern3)
    matcher.add("How", add_right, How_pattern1,How_pattern2,How_pattern3,How_pattern4,How_pattern5,
                How_pattern7,How_pattern8,How_pattern9,How_pattern10,How_pattern11,How_pattern12,How_pattern13)
    matcher.add("Who",add_label, Who_pattern1,Who_pattern2,Who_pattern3,Who_pattern4,Who_pattern5,
                Who_pattern6,Who_pattern7,Who_pattern8)
    matcher.add("What", add_right, What_pattern1,What_pattern2)
    matcher.add("Why", add_label,Why_pattern1)
    matcher.add("Mod", add_right_left,Mod_pattern1,Mod_pattern2,Mod_pattern3,Mod_pattern4)

    doc = nlp(text)
    matches = matcher(doc)
    num = 0
    start = 0 
    end = 0

    tmp_label = None


    for token in doc:
        if token._._5w1h == "Who" and re.search("VERB|ADJ",doc[token.head.i].pos_) and doc[token.head.i].i > token.i:
            tag2 = "How"
            start = token.head.i
               
            end = start + 1

            while doc[end].head.i == start and end != len(doc):

                end = end + 1

                if end == len(doc):
                    break
            
        if (re.search("ので|だから",doc[end-2:end].text) or re.search("ので、|だから、",doc[end-3:end].text)
            or re.search("ため",doc[end-1:end].text) or re.search("ため、",doc[end-2:end].text)) and doc[end].pos_ != "ADP":
            tag2 = "Why" 
            
            
        for t in doc[start:end]:
            if not t._._5w1h or t._._5w1h == "Mod" or tag2 == "Why":
                t._._5w1h = tag2


    for token in reversed(doc):

        if token._._5w1h != "Mod":
            if tmp_label == "Who" and token._._5w1h == "What":
                token._._5w1h = "Who"
            tmp_label = token._._5w1h
        else:
            token._._5w1h = tmp_label

    an_text = []
    an_label = ''
    
    for i,token in enumerate(doc):
            #print(token.ent_type)
        #an_text = an_text+token.text+'_ , '
        #an_label= an_label+token.text+'_'+token._._5w1h+', '
            #print(token.text+"\n"+token._._5w1h)
        an_text.append(token.text)
        if i+1 < len(doc):
            if (doc[i+1]._._5w1h != token._._5w1h):               
                print(''.join(an_text))
                print(token._._5w1h)
                print('\n')
                an_text = []
        else:
            print(''.join(an_text))
            print(token._._5w1h)
            print('\n')
            
    #for i,token in enumerate(doc): 
        
       # print(token.text)
       # print(token._._5w1h)

In [220]:
text = "でその後ですねまぁ実際はこのまるいちとほぼ一緒に進めることにはなるんですがあのま context 間の関連性の整理ということでまあちょっとこの後霊夢を説明しますがあの図などを使って整理することを想定しておりますがその際にちょっと既存の使用と言いますかまロジックなども参考にする必要があるのでま仕様書がない場合ってねちょうどソースを見ながらこの辺はやっていく必要があるかなって思っております。"
test_5w1h(text)

で
None


その後ですね
When


まぁ実際は
Who


このまるいちとほぼ一緒に進める
How


ことには
Who


なるんですが
How


あのまcontext間の関連性の整理ということでまあちょっとこの後
When


霊夢を
What


説明しますが
How


あの図などを使って整理することを
What


想定しておりますが
How


その際にちょっと既存の使用と言いますかまロジックなども参考にする必要が
Who


あるので
Why


ま仕様書が
Who


ない
How


場合って
Who


ねちょうどソースを
What


見ながら
How


この辺は
Who


やっていく
How


必要が
Who


あるかなって思っております。
How




In [222]:
TEST = ["でその後ですねまぁ実際はこのまるいちとほぼ一緒に進めることにはなるんですがあのま context 間の関連性の整理ということでまあちょっとこの後霊夢を説明しますがあの図などを使って整理することを想定しておりますがその際にちょっと既存の使用と言いますかまロジックなども参考にする必要があるのでま仕様書がない場合ってねちょうどソースを見ながらこの辺はやっていく必要があるかなって思っております。",
    "ちょっとこの後例を説明しますけれどもそれを一番最初にやっていくということであの必要になってくるのは以前ちょっとお願いしたんですがまあ既存の業務フローや機能一覧、今 DB の情報ですねあのこういったものを整理してインプットにしてあのドメインのメーカーかまコンテキストの明確化と言うのあのやっていくということを考えております。",
    "これが大方針大きなステップになりますがあの窓メインの明確化これ何を指してるかというのはちょっと実際にこの後ご説明しますがあのドメイン駆動設計の中ではあの境界付きコンテキストあの先日もちらっと話はしたんですがあのーコンテキストと呼ばれるですねまドメインユーザさんから見た場合はま業務領域とか事業領域とか業務範囲とかいうに考えていただければいいと思うんですがあのその範囲できちんと分けるというあの設計が必要でそのためのモデリングになります。",
    "このあとちょっとでお見せいたします。ではその中でやるま大きな四つのステップですね。",
    "ちょっとそのこの後ポイントをお話ししますけどもでもあのあまりこのモデリングそのものはですねあくまでマイクロサービスに分けるときの考え方の一つのです一つですのであのモデリングはしますがあまりここに行こう時間をかけすぎて立派なモデルの成果物を作るというよりはあのきちんとマイクロサービスを分けるためにあの中間成果物としてモデリングを行うというに考えております。",
    "で実際のちょっとへ進め方について早速ご説明したいんですが、まこの後少しお話はしますがあのままずモデリングですねマイクロサービス化に分けるためのモデリングと設計の具体的なな方法としましてあのまドメイン駆動設計ま DDD ってあの domain DRIVEN DESIGN と訳すんですがまあこれはもっともメジャーなあのマイクロサービスといえば DDD ですのでこちらでモデリングをあのすることを検討しております。",
    "今回もプロトタイプの開発においてはあのプロトタイプで開発する範囲においてあのこの一覧はお作りするしようと考えております",
    "でも同様にですねあのま上野もう一段にしただけなんですがまぁいわゆる昨日市何と言いますかマイクロサービスの単位に分けたサービス一覧というものも成果物としてあの設計書の一部して考えております。",
    "あのプロトタイプの範囲内できちんとこれを作りしようと思っています。",
    "こちらの図はですね、昨日もちらっと昨日おとといですかねあの別な打合せの際にもあの参考までにということでちょっとお出ししましたけれどもあのこれをですね最終的にはあの設計のおわりまこうかちょっとプロトタイプの範囲は限定されますのであの限定されたずにはなるかと思いますがあの当プロジェクトにおいてはですね最終的にあの設計が終わった段階でこういった図を成果物の一つとしては設計書の一つですかねあの用意するということを考えております。",
    "あのまず基本方針については以上のようなことをあげております。",
    "でまあテーブルについてはまあ DB ですね、あのサービスによってまあ分離すると。先日もちょっとあの物理的に分けるのかスキーマで分けるのかという話はありますのでそちらについてはあのまた別途最終的に検討したいと思いますがあのプロトタイプにおいては一応スキーまで分けようかなと思ってますがちょっとそれはの設計していく中でちょっとご相談させて頂こうかなと思っております。",
    "あとですね、あの業務的な観点からまあマイクロサービス化するんですが、それ以外にあの業務要件とはちょっと昨日た別な用件でですねま例えばプロトコル変換とかレイアウト変換まその他のいろいろな処理が必要な場合ってのはそれもまたサービス化するというふうに考えてます。",
    "発生した場合他の対応が必要なんですが、あのなるべく発生しないように設計するということを方針として考えております。",
    "でも欠点と言いますかまあのサービスが分かれますのであの通信時間やと分散トランザクションというものが発生する場合は処理が複雑になるってうまく欠点と言いますかデメリットがありますので、ヒトラーの後半の方で少しご説明いたしますが、あの分散トランザクションは発生なるべくしないようにあの設計するという事を一つの方針にしております。",
    "API 化してはのカプセル化することでまああの子結合とまこれ当たり前ですけどま基本的な方針でまぁ原則毎月の位置サービスというふうに考えてます。",
    "あのまあいわずもがなですがあのまあ一つのサービスでは色んなことやらせないとまああのー例えば支払いなら支払いに特化したサービスですとか、そういう風に寄せるって言う事が一つマイクロサービス化の成功のポイントになりますので、まこうぎょうしゅうとあとまあ定結合疎結合ですね。",
    "今回のま基本方針ですけれどもまあこれもよく言われることでありますが今日広告業種ですね。",
    "まああのあのマイクロサービス化によって実現することとマイ超簡単なよくネットにも出てるような内容を書いてますが、まあちょっとこちらの説明は割愛させていただきます。",
    "であの今回あの実際ですね、あのマイクロサービスを使って、マイクロサービス化でのアプリケーションを使うために作るためにはどういった手順で開発するべきかというところあの設計から開発のところをあのーままとめております。",
        "あの御社の状況もちょっとあの踏まえた上での内容にしておりますね。",
    "今画面共有されてますかね？資料は後ほどメールでも送りいたしますが、ちょっと資料10ページほどですかね、12、3ページ用意しましたので、ちょっと資料に沿ってあのー、ポイントをご説明させていただきますので、まあと資料のちょうど送りしますので詳細な点はあの見ていただいてあのー、何かご不明な点等あれば教えて頂ければなと思います。"
    "はい。あの画面共有させていただきます。",
    "ところで案を作ってまいりましたので、そこでちょっと、ディスカッションを今日ふまえてなと思うんですけども、お話をさせていただきたいと思って最後に確認事項という形で時間2時間強頂いてますけども、今こちらから依頼させて頂いてる事項、後もしもし確認事項があればそこの会話をさせていただきたいなと思っております。",
    "ズボンを掛けるバーが１つずつ動くので掛けやすいし取り出しやすいのが良いところ",
"ただ、夫は身長が高くズボンも長めのため、掛ける位置を気にしないと、床に付きます。",
"毎日日替わりで脱ぎ捨てられてソファーに重ねてかけられていた夫のズボン収納のために購入。",
"ジーンズなどの重いものを雑にかけたままで何か月も動かさずにいれば、跡がつくのではないでしょうか。",
"固定されていないので左右に動きます。",
       "犯人を見つけられず殺されるとゲームオーバー。またその日の朝からやり直し。",
       "あらすじにもある通り、ヒロインが自分が殺害される日を延々と繰り返すお話。",
       "コースによっては72分とかあるので、炊飯の時間はかかるかな。",
       "確かに横長なので、地域によっては不燃ゴミで出せないかも。",
       "釜に米がひっつく。とありましたが、確かに米はつきやすいかな。するっとは取れないですね。ちょっとひっかかる。",
       "欲を言えば、釜が軽いかな。だからあまり美味しくできなかったのかな",
       "評判を読むと、言うほど美味しくない。というコメントがありましたが、私はおいしいと思いました。",
       "値段の割に良い商品だと思います。",
       "中蓋の形がシンプルで、なべも中蓋も洗いやすく、また、メーカーの製品仕様等を調べたところ、この製品のなべの中で直接洗米してもよいと書いてあり、洗米も楽です。",
       "毎日の炊飯と、炊飯器のお手入れがとにかく楽なものを探していて、この製品に決めました。",
       "釜だけ中古品のような感じでしたので　星1つです",
       "釜だけすり替えて返品したのでは無いかと思うような商品でした",
       "日常生活の都合上、いつ車で出かけなければならないことがあるか判らない。",
       "綺麗な本体の中にキズがある釜が入っていた",
       "長らくノイズキャンセリングヘッドホンイヤホンを使ってきましたが、新参者のAppleが古参のBOSEやSONYに匹敵する商品を出したことは非常に驚きです。",
       "ノイズキャンセリング等は他のユーザーさんが評価されているので割愛します。",
       "前にAmazonで買ってレビューを書いたのに消されてる",
       "その分大きさも増したが基本的に持ち運ぶものではないので影響は少ない。",
       "手入れも洗うところ少ないし、なくなりそうな小さい部品もないし、簡単です。",
       "良くなった点は音質がよくなったところ。",
       "ので、主に音質についてのレビューです。",
       "急ぎで買った物なので返品する余裕も無く、使ってみたら釜にご飯が結構こびりつきました",]
for text in TEST:
    test_5w1h(text)
    print('\n')
    

で
None


その後ですね
When


まぁ実際は
Who


このまるいちとほぼ一緒に進める
How


ことには
Who


なるんですが
How


あのまcontext間の関連性の整理ということでまあちょっとこの後
When


霊夢を
What


説明しますが
How


あの図などを使って整理することを
What


想定しておりますが
How


その際にちょっと既存の使用と言いますかまロジックなども参考にする必要が
Who


あるので
Why


ま仕様書が
Who


ない
How


場合って
Who


ねちょうどソースを
What


見ながら
How


この辺は
Who


やっていく
How


必要が
Who


あるかなって思っております。
How




ちょっとこの後
When


例を
What


説明しますけれども
How


それを
What


一番最初にやっていくということであの必要になってくるのは以前ちょっとお願いしたんですが
How


まあ既存の業務フローや機能一覧、今
When


DBの情報ですねあのこういったものを
What


整理してインプットにしてあのドメインのメーカーかまコンテキストの明確化と言うのあのやっていくという
How


ことを
What


考えております。
How




これが
Who


大方針大きなステップになりますが
How


あの窓メインの明確化これ何を指してるかというのは
Who


ちょっと実際にこの後
When


ご説明しますが
How


あのドメイン駆動設計の中ではあの境界付きコンテキストあの先日も
When


ちらっと話は
Who


したんですが
How


あのーコンテキストと呼ばれるですねまドメインユーザさんから見た場合は
Who


ま業務領域とか事業領域とか業務範囲とかいうに考えていただければいいと思うんですが
How


あのその範囲できちんと分けるというあの設計が
Who


必要でそのためのモデリングになります。
How




このあとちょっとでお見せいたします。ではその中でやるま大きな四つのステップですね。
How




ちょっとそのこの後
When


ポイントをお話ししますけどもでもあのあまりこのモデリングそのものはです

In [223]:
text = "ところで案を作ってまいりましたので、そこでちょっと、ディスカッションを今日ふまえてなと思うんですけども、お話をさせていただきたいと思って最後に確認事項という形で時間2時間強頂いてますけども、今こちらから依頼させて頂いてる事項、後もしもし確認事項があればそこの会話をさせていただきたいなと思っております。"
test_5w1h(text)

ところで案を
What


作ってまいりましたので、
Why


そこでちょっと、ディスカッションを
What


今日
When


ふまえてなと思うんですけども、
How


お話を
What


させていただきたいと思って最後に確認事項という形で時間2時間強頂いてますけども、
How


今
When


こちらから依頼させて頂いてる事項、後もしもし確認事項が
Who


あれば
How


そこの会話を
What


させていただきたいなと思っております。
How




In [ ]:
doc[13].tag_

In [ ]:
 doc[0:5].text


In [ ]:
nlp.vocab.strings[5205465455369057791]